In [2]:
import os
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme
from langchain_nvidia_ai_endpoints._common import NVEModel

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# Function to retrieve NVIDIA API key
def retrieve_nvidia_api_key():
    api_key = os.environ.get("NVIDIA_API_KEY")
    if api_key and "nvapi-" in api_key:
        return api_key

    hard_reset = False  # Set to True if you want to reset your NVIDIA_API_KEY
    while not api_key or "nvapi-" not in api_key or hard_reset:
        try:
            assert not hard_reset
            api_key_input = input("NVIDIA API Key: ")
            assert api_key_input.startswith('nvapi-')
            api_key = api_key_input
        except:
            print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")
        hard_reset = False

    os.environ["NVIDIA_API_KEY"] = api_key
    return api_key

# Retrieve NVIDIA API key
api_key = retrieve_nvidia_api_key()
print(f"Retrieved NVIDIA_API_KEY beginning with \"{api_key[:9]}...\"")

NVEModel().available_models

NVIDIA API Key: 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
Retrieved NVIDIA_API_KEY beginning with "nvapi--ws..."


{'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_smaug_72b': '008cff6d-4f4c-4514-b61e-bcfad6ba52a7',
 'ai-gemma-2b': '04174188-f742-4069-9e72-d77c2b77d3cb',
 'ai-embed-qa-4': '09c64e32-2b65-4892-a285-2f585408d118',
 'ai-rerank-qa-mistral-4b': '0bf77f50-5c35-4488-8e7a-f49bb1974af6',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'ai-arctic-embed-l': '1528a0ad-205a-46ac-a783-94e2372586a9',
 'playground_yi_34b': '347fa3f3-d675-432c-b844-669ef8ee53df',
 'ai-recurrentgemma-2b': '2f495340-a99f-4b4b-89bd-1beb003dd896',
 'playground_nvolveqa_40k': '091a03bb-7364-4087-8090-bd71e9277520',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'playground_mistral_7b': '35ec3354-2681-4d0e-a8dd-80325dcf7c63',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'ai-phi-3-mini': '4a58c6cb-a9b4-4014-99de-3e704d4ae687',
 'playground_starcoder2_15b': '6acada03-fe2f-4e4d-9e0a-e711b9fd1b59',
 'playground_gemma_2b': '5bde

We will pull in our document index (the one we saved in the RAG agent construction notebook).
Then we will follow the steps:
- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the objective: 
Does my RAG chain outperform a narrow chatbot with limited document access?


In [5]:
##   Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder,allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout confirms that your store has been retrieved
print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"\nSample Chunk:\n\n{format_chunk(docs[len(docs)//2])}")

Constructed aggregate docstore with 542 chunks


x docstore_index/
x docstore_index/index.faiss
x docstore_index/index.pkl


Sample Chunk:

Paper: ReAct: Synergizing Reasoning and Acting in Language Models

Summary: While large language models (LLMs) have demonstrated impressive capabilities
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.
action plan generation) have primarily been studied as separate topics. In this
paper, we explore the use of LLMs to generate both reasoning traces and
task-specific actions in an interleaved manner, allowing for greater synergy
between the two: reasoning traces help the model induce, track, and update
action plans as well as handle exceptions, while actions allow it to interface
with external sources, such as knowledge bases or environments, to gather
additional information. We apply our approach, named ReAct, to a diverse set of
language and decision making tasks and demonstrate its effectiveness over
state-of-the-art baselines, as well as improved human interpretability and
trustworthiness over methods without reasoning or acting components.
Concretely, on question answering (HotpotQA) and fact verification (Fever),
ReAct overcomes issues of hallucination and error propagation prevalent in
chain-of-thought reasoning by interacting with a simple Wikipedia API, and
generates human-like task-solving trajectories that are more interpretable than
baselines without reasoning traces. On two interactive decision making
benchmarks (ALFWorld and WebShop), ReAct outperforms imitation and
reinforcement learning methods by an absolute success rate of 34% and 10%
respectively, while being prompted with only one or two in-context examples.
Project site with code: https://react-lm.github.io

Page Body: and Inner Monologue (Huang et al., 2022b), which use LLMs for robotic action planning and decision
making. In SayCan, LLMs were prompted to directly predict possible actions a robot can take, which
is then reranked by an affordance model grounded on the visual environments for ﬁnal prediction.
Inner Monologue made further improvements by adding the eponymous “inner monologue", which is
implemented as injected feedback from the environment. To our knowledge, Inner Monologue is the
ﬁrst work that demonstrates such a closed-loop system, which ReAct builds on. However, we argue
that Inner Monologue does not truly comprise of inner thoughts — this is elaborated in Section 4. We
also note that leveraging language as semantically-rich inputs in the process of interactive decision
making has been shown to be successful under other settings (Abramson et al., 2020; Karamcheti
et al., 2021; Huang et al., 2022a; Li et al., 2022). It is becoming more evident that with the help of

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr


llm = ChatNVIDIA(model='mixtral_8x7b') | StrOutputParser()
embedder = NVIDIAEmbeddings(model='nvolveqa_40k')

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string."""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
), ('user', '{input}')])

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

long_reorder = RunnableLambda(LongContextReorder().transform_documents)

context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

generator_chain = RunnableAssign({"output" : chat_prompt | llm })
generator_chain = generator_chain | output_puller 


rag_chain = retrieval_chain | generator_chain

for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

Sure, I can share something interesting from the documents you provided!

From the first document, "ReAct: Synergizing Reasoning and Acting in Language Models," there is a description of a system where a language model can both reason about a situation and take actions in that situation. This is demonstrated through an example where the language model is interacting with a simulated kitchen environment. It's interesting to consider the potential implications of such a system, which could be used in a variety of applications where both reasoning and action are important.

From the second document, "Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena," there is a discussion of the challenges of evaluating large language models (LLMs) that are capable of complex tasks such as following user instructions and answering open-ended questions. The authors note that existing benchmarks for language models are not sufficient for evaluating these more advanced capabilities, and propose a new b

In [7]:
import random

num_questions = 30
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', '{input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}", synth_questions[-1], "", sep='\n')
    pprint(synth_answers[-1], "", sep='\n')

QA Pair 1
Question: How do latent diffusion models (LDMs) improve the visual fidelity of high-resolution image synthesis 
compared to pixel-based diffusion models (DMs)?

Answer: According to the document "High-Resolution Image Synthesis with Latent Diffusion Models," LDMs apply 
diffusion models in the latent space of powerful pretrained autoencoders, reaching a near-optimal point between 
complexity reduction and detail preservation. This approach significantly boosts visual fidelity, reduces 
computational requirements, and enables powerful and flexible generators for general conditioning inputs such as 
text or bounding boxes. In contrast, pixel-based DMs typically operate directly in pixel space, consuming hundreds 
of GPU days for optimization and being expensive due to sequential evaluations. The LDMs achieve a new state of the
art for image inpainting and highly competitive performance on various tasks, including unconditional image 
generation, semantic scene synthesis, and super-resolution.

QA Pair 2
Question: How does the BERT language representation model compare to other methods in terms of natural language 
processing tasks, and how effective is it for fine-tuning and feature-based approaches?

Answer: According to the document, BERT obtains new state-of-the-art results on eleven natural language processing 
tasks, including pushing the GLUE score to 80.5%, MultiNLI accuracy to 86.7%, SQuAD v1.1 question answering Test F1
to 93.2, and SQuAD v2.0 Test F1 to 83.1. The best performing method for SQuAD v1.1 concatenates the token 
representations from the top four hidden layers of the pre-trained Transformer, which is only 0.3 F1 behind 
fine-tuning the entire model. This demonstrates that BERT is effective for both fine-tuning and feature-based 
approaches.

QA Pair 3
Question: How do the Transformer model and the ReAct approach differ in their use of language models, and what are 
some tasks where each approach excels?

...

Answer: The ReAct approach differs from other chatbots and task-oriented dialogue systems in that it explicitly 
models the thinking and reasoning procedure, and learns a policy in a cheaper way since the decision making process
only requires language description of the reasoning procedure. In contrast, other systems like BlenderBot, Sparrow,
and SimpleTOD do not explicitly consider the reasoning procedure and rely on expensive datasets and human feedback 
collections for policy learning. ReAct is also used in interactive and embodied environments for planning and 
decision making, similar to SayCan and Inner Monologue, but with a focus on using large language models for robotic
action planning and decision making.

QA Pair 29
Question: How can large language models (LLMs) be used to improve the evaluation of other LLM-based chat 
assistants, and what are some potential limitations of using LLMs as judges?

Answer: According to the paper "Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena," LLMs can be used as judges
to evaluate other LLM-based chat assistants on more open-ended questions. This approach, called "LLM-as-a-judge," 
has been shown to match both controlled and crowdsourced human preferences well, achieving over 80% agreement. This
makes it a scalable and explainable way to approximate human preferences, which are otherwise very expensive to 
obtain. The paper also proposes solutions to mitigate some of the biases and limitations of using LLMs as judges, 
such as position, verbosity, and self-enhancement biases, as well as limited reasoning ability. However, the paper 
also notes that there has not been a systematic study of the LLM-as-a-judge approach, and it examines several 
potential limitations of this approach. The MT-bench questions, 3K expert votes, and 30K conversations with human 
preferences are publicly available for further research.

QA Pair 30
Question: How does BERT, a language representation model, address the challenge of capturing the relationship 
between two sentences in tasks like Question Answering and Natural Language Inference?

Answer: BERT introduces a pre-training task called Next Sentence Prediction (NSP) to tackle this issue. The NSP 
task is designed to help the model understand the relationship between two sentences. During pre-training, BERT 
selects 15% of the token positions at random for prediction. If the i-th token is chosen, it replaces the i-th 
token with the [MASK] token, a random token, or the unchanged i-th token with certain probabilities. Then, the 
model predicts the original token using cross entropy loss. This approach helps BERT learn the relationship between
two sentences, improving its performance on downstream tasks like Question Answering and Natural Language 
Inference.

In [8]:
rag_answers = []
for i, q in enumerate(synth_questions):
    rag_answer = rag_chain.invoke(q)
    ## TODO: Compute the RAG Answer
    # rag_answer = ""
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How do latent diffusion models (LDMs) improve the visual fidelity of high-resolution image synthesis 
compared to pixel-based diffusion models (DMs)?

RAG Answer: According to the paper "High-Resolution Image Synthesis with Latent Diffusion Models," LDMs improve the
visual fidelity of high-resolution image synthesis compared to pixel-based DMs in several ways. Firstly, LDMs are 
trained in the latent space of powerful pretrained autoencoders, which allows them to reach a near-optimal point 
between complexity reduction and detail preservation. This is in contrast to previous work that trained diffusion 
models directly in pixel space, which often resulted in a trade-off between the two.

Secondly, LDMs introduce cross-attention layers into the model architecture, turning them into powerful and 
flexible generators for general conditioning inputs such as text or bounding boxes. This convolutional manner of 
high-resolution synthesis is made possible by the use of latent spaces, which enables LDMs to generate high-quality
images while significantly reducing computational requirements compared to pixel-based DMs.

Lastly, the paper notes that LDMs trained in VQ-regularized latent spaces sometimes achieve better sample quality, 
even if the reconstruction capabilities of the first stage models slightly fall behind those of their continuous 
counterparts. This suggests that LDMs are able to generate higher quality images than pixel-based DMs, even with 
some loss in reconstruction capabilities.

In summary, LDMs improve the visual fidelity of high-resolution image synthesis compared to pixel-based DMs by 
training in the latent space of powerful pretrained autoencoders, introducing cross-attention layers, and achieving
better sample quality in some cases.

QA Pair 2
Question: How does the BERT language representation model compare to other methods in terms of natural language 
processing tasks, and how effective is it for fine-tuning and feature-based approaches?

RAG Answer: The BERT language representation model is a conceptually simple and empirically powerful model that 
obtains new state-of-the-art results on eleven natural language processing tasks (Devlin et al., 2019). These tasks
include question answering, language inference, and named entity recognition, among others.

Compared to other methods, BERT is designed to pre-train deep bidirectional representations from unlabeled text by 
jointly conditioning on both left and right context in all layers (Devlin et al., 2019). This is in contrast to 
unidirectional language models and shallow concatenations of independently trained left-to-right and right-to-left 
LMs.

BERT has been shown to reduce the need for many heavily-engineered task-specific architectures, as it is the first 
fine-tuning based representation model to achieve state-of-the-art performance on a large suite of sentence-level 
and token-level tasks, outperforming many task-specific architectures (Devlin et al., 2019).

In terms of fine-tuning and feature-based approaches, BERT has certain advantages in both. For fine-tuning, BERT 
has been shown to be effective in achieving state-of-the-art results on various NLP tasks with just one additional 
output layer and without substantial task-specific architecture modifications (Devlin et al., 2019). For 
feature-based approaches, BERT allows for fixed features to be extracted from the pre-trained model, which can be 
computationally beneficial for running many experiments with cheaper models on top of this representation (Devlin 
et al., 2019).

Overall, BERT is a powerful and versatile language representation model that compares favorably to other methods in
terms of natural language processing tasks, and it is effective for both fine-tuning and feature-based approaches.

References:

* Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers 
for Language Understanding. arXiv preprint arXiv:1810.04805.
* Dai, AM, & Le, QV (2015). Semi-supervised sequence learning. Advances in neural information processing systems, 
28, 3104-3112.
* Peters, M. E., Neumann, M., Iyyer, M., Gardner, M., Clark, C., Lee, K., & Zettlemoyer, L. (2018a). Deep 
contextualized word representations. Proceedings of the 2018 Conference of the North American Chapter of the 
Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers), 
2227-2237.
* Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). Improving language understanding by generative
pre-training. arXiv preprint arXiv:1801.06146.
* Howard, J., & Ruder, S. (2018). Universal language model fine-tuning for text classification. arXiv preprint 
arXiv:1801.06146.
* Bowman, S. R., Angeli, G., Potts, C., & Manning, C. D. (2015). A large annotated corpus for learning natural 
language inference. Proceedings of the 53rd Annual Meeting of the Association for Computational Linguistics and the
7th International Joint Conference on Natural Language Processing (Volume 1: Long Papers), 2154-2164.
* Williams, A., Nangia, N., Bowman, S. R., & Subramanian, S. (2018). A broad-coverage challenge corpus for sentence
understanding through inference. Proceedings of the 2018 Conference of the North American Chapter of the 
Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers), 784-794.
* Dolan, W. B., & Brockett, C. (2005). Automatically constructing a corpus for evaluating inference in textual 
entailment. Proceedings of the 43rd Annual Meeting on Association for Computational Linguistics, 417-424.

QA Pair 3
Question: How do the Transformer model and the ReAct approach differ in their use of language models, and what are 
some tasks where each approach excels?

...

RAG Answer: The ReAct approach differs from other chatbots and task-oriented dialogue systems in the way it 
incorporates reasoning and decision making in language models. Unlike traditional chatbots and dialogue systems 
that either perform reasoning or acting in isolation, ReAct integrates both reasoning and acting in an interleaved 
manner, allowing the model to dynamically create, maintain, and adjust high-level plans for acting while 
interacting with external environments to incorporate additional information into reasoning.

ReAct prompts large language models (LLMs) to generate both verbal reasoning traces and actions pertaining to a 
task, which sets it apart from other chatbots and dialogue systems that do not explicitly model the thinking and 
reasoning procedure. This approach allows ReAct to be more flexible, performant, and robust, as it can work for 
diverse tasks with distinct action spaces and reasoning needs, and show strong generalization to new task instances
while learning solely from a few in-context examples.

Furthermore, ReAct promises an interpretable sequential decision-making and reasoning process where humans can 
easily inspect reasoning and factual correctness, and humans can control or correct the agent behavior on the go by
thought editing. This level of human alignment and controllability is not typically seen in other chatbots and 
dialogue systems.

ReAct also differs from other conversation modeling chatbots like BlenderBot and Sparrow, and task-oriented 
dialogue systems like SimpleTOD, as it explicitly considers the reasoning procedure and learns a policy in a 
cheaper way, since the decision-making process only requires language description of the reasoning procedure.

Overall, ReAct's unique approach to incorporating reasoning and decision making in language models sets it apart 
from other chatbots and task-oriented dialogue systems, making it a more flexible, performant, and human-aligned 
solution for solving diverse language reasoning and decision-making tasks.

Sources:

* ReAct: Synergizing Reasoning and Acting in Language Models, ICLR 2023.
* Shuster et al., 2022b; Glaese et al., 2022; Hosseini-Asl et al., 2020; Ahn et al., 2022; Huang et al., 2022b.

QA Pair 29
Question: How can large language models (LLMs) be used to improve the evaluation of other LLM-based chat 
assistants, and what are some potential limitations of using LLMs as judges?

RAG Answer: Large language models (LLMs) can be used to improve the evaluation of other LLM-based chat assistants 
by serving as judges to evaluate these models on more open-ended questions. This approach addresses the challenge 
of evaluating chat assistants with broad capabilities and the inadequacy of existing benchmarks in measuring human 
preferences.

To use LLMs as judges, you can implement three variations: pairwise comparison, where an LLM judge is presented 
with a question and two answers and tasked to choose the better one; self-assessment, where an LLM rates its own 
responses; and self-reflection, where an LLM compares its responses to those of another LLM. These variations can 
be implemented independently or in combination.

LLMs show potential in replacing human annotators in many tasks, and they can effectively evaluate the responses of
chat assistants and match human preferences. According to a study called "Judging LLM-as-a-Judge with MT-Bench and 
Chatbot Arena," strong LLMs can achieve an agreement rate of over 80%, which is on par with the level of agreement 
among human experts.

However, there are also potential limitations to using LLMs as judges. For example, LLMs can exhibit position bias,
verbosity bias, and self-enhancement bias, and they may have limited reasoning ability. Position bias occurs when 
an LLM prefers the first or last answer, while verbosity bias happens when an LLM favors longer answers. 
Self-enhancement bias occurs when an LLM rates its own responses higher than those of other models. Limited 
reasoning ability can also affect the performance of LLMs as judges, as they may struggle to understand complex or 
nuanced questions.

To mitigate some of these limitations, the study proposes solutions such as randomizing the order of answers, using
relative rankings instead of absolute scores, and implementing techniques to improve the reasoning ability of LLMs.

In summary, LLMs can be used to improve the evaluation of other LLM-based chat assistants by serving as judges, but
there are also potential limitations to consider, such as position bias, verbosity bias, self-enhancement bias, and
limited reasoning ability. To address these limitations, it is important to implement strategies such as 
randomizing the order of answers, using relative rankings, and improving the reasoning ability of LLMs.

QA Pair 30
Question: How does BERT, a language representation model, address the challenge of capturing the relationship 
between two sentences in tasks like Question Answering and Natural Language Inference?

RAG Answer: BERT, which stands for Bidirectional Encoder Representations from Transformers, is a language 
representation model that addresses the challenge of capturing the relationship between two sentences in tasks like
Question Answering and Natural Language Inference by using a bidirectional self-attention mechanism. This means 
that each word in the sentence is not only able to attend to all the words to its left, but also to all the words 
to its right, allowing it to capture a more comprehensive understanding of the relationship between the two 
sentences.

Additionally, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly 
conditioning on both left and right context in all layers. This allows the model to be fine-tuned with just one 
additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and
language inference, without substantial task-specific architecture modifications.

BERT obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE 
score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD 
v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point 
absolute improvement).

Reference(s):

* Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers 
for Language Understanding. arXiv preprint arXiv:1810.04805.
* The quote is from the abstract of the paper "BERT: Pre-training of Deep Bidirectional Transformers for Language 
Understanding" by Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova.

Implement a human preference metric

In [10]:
eval_instruction = """
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[0] The second answer lies, does not answer the question, or is inferior to the first answer.
[1] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification
"""

eval_prompt = ChatPromptTemplate.from_messages([
    ('system', eval_instruction), ('user', '{input}')
])

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\n{q}\n\n")

    usr_msg = f"Question: {q}\n\nAnswer 1: {a_synth}\n\n Answer 2: {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'input': usr_msg})]
    # pprint(f"Synth Answer: {a_synth}\n\n")
    # pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: How do latent diffusion models (LDMs) improve the visual fidelity of high-resolution image synthesis 
compared to pixel-based diffusion models (DMs)?

Synth Evaluation: [1] Justification: The second answer is more detailed and provides additional information about 
how latent diffusion models (LDMs) improve visual fidelity compared to pixel-based diffusion models (DMs). It 
explains the use of latent spaces, cross-attention layers, and the impact on computational requirements. 
Furthermore, it discusses the benefits of training LDMs in VQ-regularized latent spaces. The second answer does not
introduce any inconsistencies and provides a better understanding of the comparison between LDMs and pixel-based 
DMs.

Set 2

Question: How does the BERT language representation model compare to other methods in terms of natural language 
processing tasks, and how effective is it for fine-tuning and feature-based approaches?

Synth Evaluation: [0] Justification: Although the second answer provides a good overview of BERT and its 
performance on various NLP tasks, it does not directly compare BERT to other methods in terms of natural language 
processing tasks, nor does it discuss the effectiveness of BERT for fine-tuning and feature-based approaches as 
thoroughly as the first answer. The first answer is more specific and detailed in addressing the question.

Set 3

Question: How do the Transformer model and the ReAct approach differ in their use of language models, and what are 
some tasks where each approach excels?

...

Synth Evaluation: [1] The second answer is better than the first and does not introduce any inconsistencies.

Justification: The second answer provides a more detailed explanation of how the ReAct approach differs from other 
chatbots and task-oriented dialogue systems in terms of incorporating reasoning and decision making in language 
models. It highlights the unique features of ReAct, such as its ability to dynamically create, maintain, and adjust
high-level plans for acting while interacting with external environments, and its promise of an interpretable 
sequential decision-making and reasoning process. The second answer also acknowledges the limitations of other 
chatbots and dialogue systems and explains how ReAct addresses those limitations. Overall, the second answer 
provides a more comprehensive and informative response to the question.

Set 29

Question: How can large language models (LLMs) be used to improve the evaluation of other LLM-based chat 
assistants, and what are some potential limitations of using LLMs as judges?

Synth Evaluation: [1] The second answer is better than the first because it provides a more detailed explanation of
how LLMs can be used as judges to evaluate other LLM-based chat assistants, including the three variations of 
pairwise comparison, self-assessment, and self-reflection. The second answer also includes additional information 
about the potential limitations of using LLMs as judges and proposes solutions to mitigate these limitations. The 
second answer is consistent with the first answer and does not introduce any inconsistencies.

Set 30

Question: How does BERT, a language representation model, address the challenge of capturing the relationship 
between two sentences in tasks like Question Answering and Natural Language Inference?

Synth Evaluation: [1] The second answer is better than the first and does not introduce any inconsistencies.

Justification:

Answer 2 provides a more comprehensive explanation of how BERT addresses the challenge of capturing the 
relationship between two sentences. It explains the bidirectional self-attention mechanism, which allows each word 
in the sentence to attend to all the words to its left and right, capturing a more comprehensive understanding of 
the relationship between the two sentences. Additionally, it discusses BERT's pre-training process and its impact 
on various natural language processing tasks. This information is not present in Answer 1, making Answer 2 more 
informative and valuable to the user.

Note as we use llm as a judge, the final score may vary when you execute the pipeline every time. But overall the RAG pipeline should always outperform the basic one.

In [14]:
pref_score = sum(("[1]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.8666666666666667
